In [1]:
import os
import ipywidgets as widgets

In [2]:
from projector import project  #, run_projection (não consegui usar assim) 

import copy
# import os
from time import perf_counter

import click
import imageio
import numpy as np
import PIL.Image
import torch
import torch.nn.functional as F

import dnnlib
import legacy

def run_projection(
    network_pkl: str,
    target_fname: str,
    outdir: str,
    save_video: bool,
    seed: int,
    num_steps: int
):
    """Project given image to the latent space of pretrained network pickle.

    Examples:

    \b
    python projector.py --outdir=out --target=~/mytargetimg.png \\
        --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl
    """
    np.random.seed(seed)
    torch.manual_seed(seed)

    # Load networks.
    print('Loading networks from "%s"...' % network_pkl)
    device = torch.device('cuda')
    with dnnlib.util.open_url(network_pkl) as fp:
        G = legacy.load_network_pkl(fp)['G_ema'].requires_grad_(False).to(device) # type: ignore

    # Load target image.
    target_pil = PIL.Image.open(target_fname).convert('RGB')
    w, h = target_pil.size
    s = min(w, h)
    target_pil = target_pil.crop(((w - s) // 2, (h - s) // 2, (w + s) // 2, (h + s) // 2))
    target_pil = target_pil.resize((G.img_resolution, G.img_resolution), PIL.Image.LANCZOS)
    target_uint8 = np.array(target_pil, dtype=np.uint8)

    # Optimize projection.
    start_time = perf_counter()
    projected_w_steps = project(
        G,
        target=torch.tensor(target_uint8.transpose([2, 0, 1]), device=device), # pylint: disable=not-callable
        num_steps=num_steps,
        device=device,
        verbose=True
    )
    print (f'Elapsed: {(perf_counter()-start_time):.1f} s')

    # Render debug output: optional video and projected image and W vector.
    os.makedirs(outdir, exist_ok=True)
    if save_video:
        video = imageio.get_writer(f'{outdir}/proj.mp4', mode='I', fps=10, codec='libx264', bitrate='16M')
        print (f'Saving optimization progress video "{outdir}/proj.mp4"')
        for projected_w in projected_w_steps:
            synth_image = G.synthesis(projected_w.unsqueeze(0), noise_mode='const')
            synth_image = (synth_image + 1) * (255/2)
            synth_image = synth_image.permute(0, 2, 3, 1).clamp(0, 255).to(torch.uint8)[0].cpu().numpy()
            video.append_data(np.concatenate([target_uint8, synth_image], axis=1))
        video.close()

    # Save final projected frame and W vector.
    target_pil.save(f'{outdir}/target.png')
    projected_w = projected_w_steps[-1]
    synth_image = G.synthesis(projected_w.unsqueeze(0), noise_mode='const')
    synth_image = (synth_image + 1) * (255/2)
    synth_image = synth_image.permute(0, 2, 3, 1).clamp(0, 255).to(torch.uint8)[0].cpu().numpy()
    PIL.Image.fromarray(synth_image, 'RGB').save(f'{outdir}/proj.png')
    np.savez(f'{outdir}/projected_w.npz', w=projected_w.unsqueeze(0).cpu().numpy())


## titulo seção

Upload da foto

In [3]:
uploader = widgets.FileUpload()
uploader

FileUpload(value={}, description='Upload')

Mostrar imagem

In [21]:
img = widgets.Image(value=uploader.data[0])
img

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\x1a\x00\x00\x02Q\x08\x06\x00\x00\x00\x03&\x1e\x8…

Salvar imagem

In [22]:
upload_dir = '_my_images/005_moana_100'
os.makedirs(upload_dir, exist_ok=True)
uploaded_fname = f'{upload_dir}/foto_real.png'

with open(uploaded_fname, "wb") as fp:
    fp.write(uploader.data[0])

In [27]:
network_pkl = 'https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl'
target_fname = uploaded_fname
outdir = upload_dir
save_video = True
seed = 42
num_steps = 1000
        
run_projection(network_pkl, target_fname, outdir, save_video, seed, num_steps)

Loading networks from "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl"...
Computing W midpoint and stddev using 10000 samples...
step    1/1000: dist 0.68 loss 9307.49
step    2/1000: dist 0.67 loss 12018.04
step    3/1000: dist 0.68 loss 11683.14
step    4/1000: dist 0.70 loss 11050.48
step    5/1000: dist 0.68 loss 10178.36
step    6/1000: dist 0.67 loss 9129.27
step    7/1000: dist 0.71 loss 7964.16
step    8/1000: dist 0.68 loss 6746.20
step    9/1000: dist 0.73 loss 5539.89
step   10/1000: dist 0.63 loss 4411.89
step   11/1000: dist 0.70 loss 3439.56
step   12/1000: dist 0.61 loss 2676.20
step   13/1000: dist 0.65 loss 2111.94
step   14/1000: dist 0.67 loss 1698.43
step   15/1000: dist 0.65 loss 1403.90
step   16/1000: dist 0.66 loss 1214.30
step   17/1000: dist 0.67 loss 1104.83
step   18/1000: dist 0.67 loss 1097.09
step   19/1000: dist 0.62 loss 1190.42
step   20/1000: dist 0.68 loss 1370.04
step   21/1000: dist 0.67 loss 1621.54
step   22/1000: dist

In [26]:
a = 2

In [ ]:
!python generate.py --outdir=out --projected_w=out/projected_w.npz \
  --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl


Paper ada
- https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/ada-paper.pdf

Consultas interessantes:
- https://realpython.com/primer-on-python-decorators/#decorating-functions-with-arguments (retomar dessa seção)

- https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html
- https://stackoverflow.com/questions/63215752/how-to-use-fileupload-widget-in-jupyter-lab
- https://pillow.readthedocs.io/en/stable/reference/Image.html


Curiosidades:
- https://thispersondoesnotexist.com/
- https://noticias.r7.com/tecnologia-e-ciencia/fotos/site-apavorante-cria-rostos-de-pessoas-que-nunca-existiram-21022019
